In [1]:
from utils.hls import HLSBand
from utils.hls import HLSCatalog
from utils.hls import HLSTileLookup
from utils.hls import fia_csv_to_data_catalog_input

In [2]:
lookup = HLSTileLookup()

Reading tile extents...
Read tile extents for 56686 tiles


In [3]:
bands = [
    HLSBand.COASTAL_AEROSOL,
    HLSBand.BLUE,
    HLSBand.GREEN,
    HLSBand.RED,
    HLSBand.NIR_NARROW,
    HLSBand.SWIR1,
    HLSBand.SWIR2,
    HLSBand.CIRRUS,
    HLSBand.QA,
]

In [ ]:
point_catalog = HLSCatalog.from_point_pandas(df=fia_csv_to_data_catalog_input('./fia_10.csv'), bands=bands, tile_lookup=lookup)

In [ ]:
point_catalog.to_zarr('fia10.zarr')

In [3]:
pc = HLSCatalog.from_zarr('fia10.zarr')

In [4]:
%%time

# 2015-2019 USA
bbox = [-124.98046874999999, 24.367113562651262, -66.70898437499999, 49.49667452747045]
years = [2015, 2016, 2017, 2018, 2019]
bbox_catalog = HLSCatalog.from_bbox(bbox, years, bands, lookup)

CPU times: user 47min 46s, sys: 15.4 s, total: 48min 2s
Wall time: 1h 12min 39s


In [5]:
bbox_catalog.to_zarr("usa_2015-2019.zarr")